This notebook is to test the filter changer timing and repeatability.  In particular:

The requirement: LVV-14633 LTS-508-REQ-0015-V-01: Filter Changing_1
* Filter changer must be able to switch between filters remotely
* Positioning requirement is given in requirement LTS-508-4
* Filter change time: 30 s maximum
* Repeatability of motion: < +/- 0.1 mm (TBR)
* Filter installation positioning and reconfiguration: +/- 0.1 mm lateral, +/- 2.6 arcmin for rotation. (verified elsewhere)

In [ ]:
import asyncio
import logging
import sys
import os

from astropy.coordinates import Angle
from astropy.time import Time
import astropy.units as u

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel.mtcs import MTCS
from lsst.ts.observatory.control.maintel.comcam import ComCam

In [ ]:
print(os.environ["OSPL_URI"])

In [ ]:
os.environ["LSST_DDS_RESPONSIVENESS_TIMEOUT"]="15s"

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()

await asyncio.sleep(10)

comcam = ComCam(domain)
await asyncio.gather(comcam.start_task)

In [ ]:
await comcam.enable(overrides={"cccamera": "Normal",
                              "ccheaderservice": None})

In [ ]:
# note that we are slightly out of spec for moving positions 0 to 2 and 2 to 0 by 5.3 seconds.  
# This might be due to the timing measurement.  Can we get get better measurments from the EFD?
# also need to grab the positions from the linear encoder to calculate the repeatability.

for i in range(0,10):
    #get start time
    startdate = Time.now()
    #change filter
    await comcam.setup_instrument(filter=filters[i % 3])
    #get end time
    enddate = Time.now()
    print("Movement "+str(i)+" duration: "+str(24*60*60*(enddate-startdate)))
for i in range(0,11):
    #get start time
    startdate = Time.now()
    #change filter
    await comcam.setup_instrument(filter=filters[(10-i) % 3])
    #get end time
    enddate = Time.now()
    print("Movement "+str(10+i)+" duration: "+str(24*60*60*(enddate-startdate)))

In [ ]:
await comcam.standby()

In [ ]:
await domain.close()

In [ ]:
## if state history was lost
await comcam.rem.cccamera.cmd_start.set_start(configurationOverride="Normal",timeout=10)

In [ ]:
await comcam.rem.cccamera.cmd_enterControl.start(timeout=10)

In [ ]:
await comcam.rem.cccamera.cmd_enable.start(timeout=10)